# Wprowadzenie do `jinja2`

Biblioteka `jinj2` jest silnikiem szablonów napisanym Pythonie. Do czego służy takie silnik szablonów? Ogólnie rzecz ujmując taki silnik wykorzystujemy, kiedy chcemy dynamicznie generować jakiegoś rodzaju dokumenty. Silnik `jinja2` powstał do dynamicznego generowania dokumentów HTML (stron internetowych), ale można go wykorzystać do generowania dowolnych dokumentów, które mają postać tekstu. Na przykład silnik  ten używany jest do przetwarzania plików konfiguracyjnych w pakiecie Ansible (https://www.ansible.com/), służącym do automatycznego zarządzadzania dużą liczbą komputerów (zazwyczaj stacji roboczych lub serwerów). Silnik ten jest więc bardzo elastyczny. Z powodzeniem można go stosować np. w połączeniu z LaTeXem (http://eosrei.net/articles/2015/11/latex-templates-python-and-jinja2-generate-pdfs) do automatycznego generowania dokumentów PDF. Dla nas silnik ten jest interesujący, ponieważ możemy użyć go do:
+ generowania raportów z eksperymentu lub ze zbierania danych badawczych
+ generowania kwestionariuszy do użycia w badaniach

Obie te rzeczy można wykonać na mnóstwo innych sposobów, ale jeśli zależy nam na elastyczności i dużych możliwościach to `jinja2` (lub dowolny inny silnik tego rodzaju) jest świetnym rozwiązaniem.

Z biblioteki `jinja` zaimportujemy dwie klasy: `Environment` oraz `Template`.

In [6]:
from jinja2 import Template, Environment

Najprostszy przykład może wydawać się dość trywialny, ale dobrze ilustruje "o co chodzi" z szablonami. Najpierw stworzymy szablon (w naszym wypadku będzie to po prostu łańcuch znaków z odpowiednią składnią), skonstruujemy odpowiedni obiekt i potem wypełnimy go danymi:

In [12]:
template_string = "Prowadzący kurs to {{ firstname }} {{ lastname }}"
template = Template(template_string) 
template.render(firstname='Bartosz', lastname='Maćkiewicz') # render wypełnia szablon danymi

'Prowadzący kurs to Bartosz Maćkiewicz'

Bardziej skomplikowany przykład byłby taki: załóżmy, że mamy pythonowską listę z uczestnikami kursu i chcielibyśmy wygenerować ich listę jako plik tekstowy. Moglibyśmy oczywiście zrobić to za pomocą Pythona dodając do siebie stringi, ale rozwiązanie takie na dłuższą metę trudno utrzymać w porządku.

In [14]:
uczestnicy_kursu = [{'imię' : 'Jan',
                     'nazwisko' : 'Kowalski',
                     'kierunek' : 'kognitywistyka'},
                    {'imię' : 'Piotr',
                     'nazwisko' : 'Nowak',
                     'kierunek' : 'filozofia'},
                    {'imię' : 'Anna',
                     'nazwisko' : 'Jastrzębska',
                     'kierunek' : None},
                   ]

Stwórzmy więc szablon w języku szablonów `jinja2`. Poniższy szablon ilustruje większość ważnych elementów składni tego języka:
+ `{{ zmienna }}` zastępowane jest przez wartość zmiennej `zmienna`
+ `{% for ... in ... %} wyrażenie {% endfor %}` wykonuje wyrażenie dla każdej wartości z obiektu, po którym iterujemy
+ `{% if warunek %} wyrazenie1 [{% else %} wyrazenie2 ] {% endif %}` - zastępuje `wyrazenie1` wartością zmiennej jeśli `warunek` jest spełniony, w innym wypadku robi to z `wyrazenie2` (`else` jest nieobowiązkowe)
+ `zmienna|filtr` - stosuje filtr do zmiennej, w tym wypadku filtr `length` zastępuje zmienną jej długością

In [22]:
template_string = '''Liczba uczestników: {{ participants|length }}

{% for participant in participants %}
Dane uczestnika:
{{ participant.imię }} {{ participant.nazwisko }}
kierunek: {% if participant.kierunek is not none  %} {{ participant.kierunek }} {% else %} brak danych {% endif %}
{% endfor %}
'''

template = Template(template_string)
print(template.render(participants = uczestnicy_kursu))

Liczba uczestników: 3


Dane uczestnika:
Jan Kowalski
kierunek:  kognitywistyka 

Dane uczestnika:
Piotr Nowak
kierunek:  filozofia 

Dane uczestnika:
Anna Jastrzębska
kierunek:  brak danych 



Może zdarzyć się, że w jakimś rodzaju dokumentów pewne kombinacje znaków są zabronione. Domyślna składnia `jinja2` jest dobra, jeśli pracujemy z plikami HTML/XML, nic nie stoi na przeszkodzie, żeby zmienić łańcuchy znaków, które rozpoczynają lub kończą różne konstrukcje dostępne w `jinja2`. W poniższym przypadku zastąpimy różnego rodzaju nawiasy klamrowe innymi ciągami znaków.

In [23]:
env = Environment(variable_start_string = 'XX', # zastępujemy {{
                 variable_end_string = 'XX', # zastępujemy }}
                 block_start_string = 'XV', # zastępujemy {%
                 block_end_string = 'VX') # zastępujemy %}

template_string = '''Liczba uczestników: XX participants|length XX

XV for participant in participants VX
Dane uczestnika:
XX participant.imię XX XX participant.nazwisko XX
kierunek: XV if participant.kierunek is not none  VX XX participant.kierunek XX XV else VX brak danych XV endif VX
XV endfor VX
'''
template = env.from_string(template_string)
print(template.render(participants = uczestnicy_kursu))

Liczba uczestników: 3


Dane uczestnika:
Jan Kowalski
kierunek:  kognitywistyka 

Dane uczestnika:
Piotr Nowak
kierunek:  filozofia 

Dane uczestnika:
Anna Jastrzębska
kierunek:  brak danych 



Możliwości, jakie daje silnik szablonów są bardzo duże. Zachęcam do eksperymentowania - to jedno z najlepszych narzędzi "ułatwiających życie"!